In [10]:
%pip install onnx


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import os
import json
import numpy as np
import tensorflow as tf
import onnx

In [12]:


# Save tensor as binary file
def save_tensor_as_bin(tensor, filepath):
    np_array = tensor.numpy() if isinstance(tensor, tf.Tensor) else tensor
    np_array.tofile(filepath)

# Save JSON file
def save_json(data, filepath):
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=4)

# Create directories
def create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [13]:
# Layer-wise dump with a single JSON for the model architecture
def layer_wise_dump_tf(model, input_tensor, save_dir):
    # Ensure save directory structure
    create_dir(save_dir)
    create_dir(f"{save_dir}/input")
    create_dir(f"{save_dir}/output")
    create_dir(f"{save_dir}/weights")
    create_dir(f"{save_dir}/reference")

    model_architecture = {
        "model_name": model.name,
        "num_layers": len(model.layers),
        "input_shape": list(model.input_shape) if hasattr(model, 'input_shape') else "Unknown",
        "output_shape": list(model.output_shape) if hasattr(model, 'output_shape') else "Unknown",
        "layers": []
    }

    x = input_tensor
    for layer in model.layers:
        layer_name = layer.name

        # Save input tensor
        input_filepath = f"{save_dir}/input/{layer_name}_input.bin"
        save_tensor_as_bin(x, input_filepath)

        # Save weights
        weight_filepaths = []
        if hasattr(layer, 'weights') and layer.weights:
            for weight in layer.weights:
                weight_name = weight.name.split("/")[-1].replace(":", "_")
                weight_filepath = f"{save_dir}/weights/{layer_name}_{weight_name}.bin"
                save_tensor_as_bin(weight.numpy(), weight_filepath)
                weight_filepaths.append(weight_filepath)

        # Compute layer output
        x = layer(x)

        # Save output tensor
        output_filepath = f"{save_dir}/output/{layer_name}_output.bin"
        save_tensor_as_bin(x, output_filepath)

        # Gather layer attributes
        layer_attributes = {
            "layer_name": layer_name,
            "type": layer.__class__.__name__,
            "input_shape": list(layer.input_shape) if hasattr(layer, 'input_shape') else "Unknown",
            "output_shape": list(layer.output_shape) if hasattr(layer, 'output_shape') else "Unknown",
            "input_file": input_filepath,
            "output_file": output_filepath,
            "weight_files": weight_filepaths,
        }
        if hasattr(layer, 'kernel_size'):
            layer_attributes["kernel_size"] = layer.kernel_size
        if hasattr(layer, 'strides'):
            layer_attributes["strides"] = layer.strides
        if hasattr(layer, 'padding'):
            layer_attributes["padding"] = layer.padding
        if hasattr(layer, 'activation') and layer.activation:
            layer_attributes["activation"] = layer.activation.__name__

        # Add layer attributes to the model architecture JSON
        model_architecture["layers"].append(layer_attributes)

    # Save the complete model architecture to a single JSON file
    save_json(model_architecture, f"{save_dir}/model_architecture.json")

    return model_architecture


In [14]:
model = tf.keras.models.load_model("D:\MCW\Assignment-1\DATA_AUG.keras")

<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\M'
C:\Users\msrak\AppData\Local\Temp\ipykernel_15164\4102964686.py:1: SyntaxWarning: invalid escape sequence '\M'
  model = tf.keras.models.load_model("D:\MCW\Assignment-1\DATA_AUG.keras")


In [15]:
input_tensor = tf.random.normal([1, 32, 32, 3])  # Random input for CIFAR-10
layer_wise_dump_tf(model, input_tensor, save_dir="data")

{'model_name': 'sequential',
 'num_layers': 11,
 'input_shape': [None, 32, 32, 3],
 'output_shape': [None, 10],
 'layers': [{'layer_name': 'conv2d',
   'type': 'Conv2D',
   'input_shape': 'Unknown',
   'output_shape': 'Unknown',
   'input_file': 'data/input/conv2d_input.bin',
   'output_file': 'data/output/conv2d_output.bin',
   'weight_files': ['data/weights/conv2d_kernel.bin',
    'data/weights/conv2d_bias.bin'],
   'kernel_size': (3, 3),
   'strides': (1, 1),
   'padding': 'same',
   'activation': 'linear'},
  {'layer_name': 'activation',
   'type': 'Activation',
   'input_shape': 'Unknown',
   'output_shape': 'Unknown',
   'input_file': 'data/input/activation_input.bin',
   'output_file': 'data/output/activation_output.bin',
   'weight_files': [],
   'activation': 'relu'},
  {'layer_name': 'max_pooling2d',
   'type': 'MaxPooling2D',
   'input_shape': 'Unknown',
   'output_shape': 'Unknown',
   'input_file': 'data/input/max_pooling2d_input.bin',
   'output_file': 'data/output/max_po

In [16]:
import os
import json
import numpy as np

def generate_layer_config(model, save_dir, config_file_path):
    # Build model or run a dummy forward pass to infer shapes
    if not model.built:
        # Infer the input shape from the model's first layer
        input_shape = model.input_shape if hasattr(model, 'input_shape') else None
        if input_shape:
            model.build(input_shape)
        else:
            # Use dummy input if input_shape is unknown
            dummy_input = np.random.random((1, *model.layers[0].input_shape[1:]))
            model(dummy_input)
    
    layer_configs = []  # List to store layer-wise configurations

    for layer in model.layers:
        layer_name = layer.name

        # Construct layer configuration
        layer_config = {
            "layer_name": layer_name,
            "type": layer.__class__.__name__,  # Correct way to get the class name
            "input_file_path": f"{save_dir}/input/{layer_name}_input.bin",
            "output_file_path": f"{save_dir}/output/{layer_name}_output.bin",
            "weights_file_paths": [],  # To be filled if the layer has weights
            "attributes": {
                "input_shape": list(layer.input_shape) if hasattr(layer, 'input_shape') else "Unknown",
                "output_shape": list(layer.output_shape) if hasattr(layer, 'output_shape') else "Unknown",
            }
        }

        # Add weights and biases if applicable
        if hasattr(layer, 'weights') and layer.weights:
            for weight in layer.weights:
                weight_name = weight.name.split("/")[-1].replace(":", "_")
                weight_file_path = f"{save_dir}/weights/{layer_name}_{weight_name}.bin"
                layer_config["weights_file_paths"].append(weight_file_path)

        # Add other layer-specific attributes if available
        if hasattr(layer, 'kernel_size'):
            layer_config["attributes"]["kernel_size"] = layer.kernel_size
        if hasattr(layer, 'strides'):
            layer_config["attributes"]["strides"] = layer.strides
        if hasattr(layer, 'padding'):
            layer_config["attributes"]["padding"] = layer.padding
        if hasattr(layer, 'activation') and layer.activation:
            layer_config["attributes"]["activation"] = layer.activation.__name__

        # Append this layer's config to the list
        layer_configs.append(layer_config)

    # Save all configurations to a JSON file
    with open(config_file_path, 'w') as config_file:
        json.dump({"layers": layer_configs}, config_file, indent=4)

    print(f"Configuration file saved to {config_file_path}")


In [17]:
# Assuming the `data` folder is in the current directory
data_folder = "D:/MCW/Assignment-1/data"
output_json_file = "D:/MCW/Assignment-1/model_config.json"
model = tf.keras.models.load_model("D:\MCW\Assignment-1\DATA_AUG.keras")

generate_layer_config(model, data_folder, output_json_file)


<>:4: SyntaxWarning: invalid escape sequence '\M'
<>:4: SyntaxWarning: invalid escape sequence '\M'
C:\Users\msrak\AppData\Local\Temp\ipykernel_15164\1512840883.py:4: SyntaxWarning: invalid escape sequence '\M'
  model = tf.keras.models.load_model("D:\MCW\Assignment-1\DATA_AUG.keras")


Configuration file saved to D:/MCW/Assignment-1/model_config.json


In [18]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     4,194,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,269,536 (50.62 MB)

 Trainable params: 4,423,178 (16.87 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 8,846,358 (33.75 MB)